<a href="https://colab.research.google.com/github/dspacks/MySchoolMenuNotebook/blob/main/school_menus_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# School Menu Calendar Generator

**Simple approach:** Just paste the URL from your browser!

## How to Use

1. **Run the first cell** to install/import libraries
2. **Copy a menu URL from your browser** and paste it in "Step 1: Enter Menu URLs"
3. **Run remaining cells** to generate calendars

### Where to Find URLs

1. Visit your school menu website (e.g., healthepro.com or myschoolmenus.com)
2. Navigate to your school
3. Click on a menu (Breakfast, Lunch, etc.)
4. Copy the URL from your browser address bar
5. Paste it below!

### Example URLs

- `https://menus.healthepro.com/organizations/2230/sites/14066/menus/102948` (HealthePro)
- `https://www.myschoolmenus.com/organizations/2230/sites/14066/menus/65638` (MySchoolMenus)

In [99]:
# Install required library
!pip install requests

In [100]:
# Import required modules
import requests
import json
import re
import uuid
import logging
from datetime import datetime, time, date, timedelta
from dataclasses import dataclass, field
from enum import Enum
from pathlib import Path
from typing import List, Dict, Optional

logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

print("✓ Libraries imported")

✓ Libraries imported


## Helper Classes and Functions

In [101]:
def parse_menu_url(url: str) -> Dict[str, int]:
    """
    Extract district_id, site_id, menu_id from menu URL.

    Example:
        parse_menu_url("https://menus.healthepro.com/organizations/2230/sites/14066/menus/102948")
        # Returns: {'district_id': 2230, 'site_id': 14066, 'menu_id': 102948}
    """
    url = url.strip()
    pattern = r'/organizations/(\d+)/sites/(\d+)/menus/(\d+)'
    match = re.search(pattern, url)

    if not match:
        raise ValueError(f"Could not parse URL: {url}")

    district_id, site_id, menu_id = match.groups()
    return {
        'district_id': int(district_id),
        'site_id': int(site_id),
        'menu_id': int(menu_id)
    }

# Test the parser
test_url = "https://menus.healthepro.com/organizations/2230/sites/14066/menus/102948"
result = parse_menu_url(test_url)
print(f"✓ URL Parser Working")
print(f"  Example: {result}")

✓ URL Parser Working
  Example: {'district_id': 2230, 'site_id': 14066, 'menu_id': 102948}


In [102]:
class MenuType(Enum):
    BREAKFAST = "breakfast"
    LUNCH = "lunch"
    SNACK = "snack"
    DINNER = "dinner"

@dataclass
class MenuItem:
    date: date
    main_dish: str
    categories: List[str] = field(default_factory=list)
    all_items: List[str] = field(default_factory=list)
    menu_type: MenuType = MenuType.LUNCH

@dataclass
class CalendarEvent:
    summary: str
    dtstart: datetime | date
    uid: str = field(default_factory=lambda: str(uuid.uuid4()))
    dtstamp: datetime = field(default_factory=datetime.now)
    description: str = ""
    dtend: Optional[datetime] = None
    transp: str = "OPAQUE"

    def to_ical_lines(self) -> List[str]:
        def escape_text(text: str) -> str:
            text = text.replace("\\", "\\\\")
            text = text.replace(",", "\\,")
            text = text.replace(";", "\\;")
            text = text.replace("\n", "\\n")
            text = text.replace("\r", "\\r")
            return text

        def fold_line(property_name: str, content: str, max_length: int = 75) -> List[str]:
            first_line = f"{property_name}:{content}"
            if len(first_line) <= max_length:
                return [first_line]
            lines = []
            current_line = first_line
            while len(current_line) > max_length:
                lines.append(current_line[:max_length])
                current_line = " " + current_line[max_length:]
            if current_line:
                lines.append(current_line)
            return lines

        lines = [
            "BEGIN:VEVENT",
            f"SUMMARY:{escape_text(self.summary)}",
        ]

        if isinstance(self.dtstart, datetime):
            lines.append(f"DTSTART:{self.dtstart.strftime('%Y%m%dT%H%M%S')}")
        else:
            lines.append(f"DTSTART;VALUE=DATE:{self.dtstart.strftime('%Y%m%d')}")

        if self.dtend:
            lines.append(f"DTEND:{self.dtend.strftime('%Y%m%dT%H%M%S')}")

        lines.append(f"DTSTAMP:{self.dtstamp.strftime('%Y%m%dT%H%M%SZ')}")
        lines.append(f"UID:{self.uid}")

        if self.description:
            desc_lines = fold_line("DESCRIPTION", escape_text(self.description))
            lines.extend(desc_lines)

        lines.append(f"TRANSP:{self.transp}")
        lines.append("END:VEVENT")
        return lines

print("✓ Data classes defined")

✓ Data classes defined


In [103]:
class MenuClient:
    """API client that fetches menus from HealthePro or MySchoolMenus."""

    def __init__(self):
        self.base_urls = [
            "https://menus.healthepro.com/api",
            "https://www.myschoolmenus.com/api",
            "https://myschoolmenus.com/api"
        ]

    def fetch_menu(self, url: str, menu_type: MenuType = MenuType.LUNCH) -> List[MenuItem]:
        """Fetch menu items from URL."""
        ids = parse_menu_url(url)
        logger.info(f"Fetching: district {ids['district_id']}, site {ids['site_id']}, menu {ids['menu_id']} from {url}...")

        # Get current year and month for the new endpoint
        today = datetime.now()
        year = today.year
        month = today.month

        # --- Modified endpoint to fetch daily menu items with year, month, and date_overwrites ---
        endpoint = f"/organizations/{ids['district_id']}/menus/{ids['menu_id']}/year/{year}/month/{month}/date_overwrites"
        # ------------------------------------------------------------------------------------------

        data = None
        last_error = None
        for base_url in self.base_urls:
            try:
                full_url = f"{base_url}{endpoint}"
                logger.info(f"  Trying {full_url}")
                response = requests.get(full_url, timeout=10)
                response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
                data = response.json()
                logger.info(f"  Successfully fetched data from {base_url}")
                logger.warning(f"  Raw API data received (first 500 chars): {str(data)[:500]}...") # Existing logging line
                break
            except requests.exceptions.RequestException as e:
                last_error = e
                logger.warning(f"  Failed to fetch from {base_url}: {e}")
                continue
            except json.JSONDecodeError as e:
                last_error = e
                logger.warning(f"  Failed to decode JSON from {base_url}: {e}")
                continue
        else:
            raise Exception(f"Failed to fetch menu from all base URLs: {last_error}")

        if not data:
            logger.warning(f"  No data received for {url}")
            return []

        # --- Improved logic to correctly extract daily entries, prioritizing raw_api_response['data'] if it's a list ---
        raw_api_response = data # This is the full JSON response, could be a list or a dict
        actual_entries = []

        if isinstance(raw_api_response, dict) and 'data' in raw_api_response and isinstance(raw_api_response['data'], list):
            # Prioritize: 'data' key contains a list directly
            actual_entries = raw_api_response['data']
            logger.info("    Found daily items under 'data' key (list).")
        elif isinstance(raw_api_response, list):
            # Case 1: API returns a list directly
            actual_entries = raw_api_response
            logger.info("    API response is a list of entries directly.")
        elif isinstance(raw_api_response, dict):
            # Case 2: API returns a dictionary, check its 'data' key (if not already handled)
            if 'data' in raw_api_response:
                data_value = raw_api_response['data']
                if isinstance(data_value, dict):
                    # Case 2b: 'data' key contains a dict (e.g., metadata), check for a list within it
                    if 'items' in data_value and isinstance(data_value['items'], list):
                        actual_entries = data_value['items']
                        logger.info("    Found daily items under 'data.items' key.")
                    elif 'days' in data_value and isinstance(data_value['days'], list):
                        actual_entries = data_value['days']
                        logger.info("    Found daily items under 'data.days' key.")
                    else:
                        logger.warning(f"    API response 'data' is a dictionary, but no 'items' or 'days' list found within it. Cannot extract daily menu entries. Raw data.data: {data_value}")
                else:
                    logger.warning(f"    Unexpected type for 'data' key value in API response ({type(data_value)}). Expected list or dictionary. Raw data.data: {data_value}")
            else:
                # Case 2c: API returns a dictionary, but no 'data' key, check for 'items' or 'days' directly
                if 'items' in raw_api_response and isinstance(raw_api_response['items'], list):
                    actual_entries = raw_api_response['items']
                    logger.info("    Found daily items under 'items' key (top level)."
)
                elif 'days' in raw_api_response and isinstance(raw_api_response['days'], list):
                    actual_entries = raw_api_response['days']
                    logger.info("    Found daily items under 'days' key (top level).")
                else:
                    logger.warning(f"    API response is a dictionary, but no 'data', 'items', or 'days' list found at top level. Cannot extract daily menu entries. Raw data: {raw_api_response}")
        else:
            logger.warning(f"    Unexpected type for API response ({type(raw_api_response)}). Expected list or dictionary. Raw data: {raw_api_response}")
        # ----------------------------------------------------------------

        items = []
        logger.info(f"  Processing {len(actual_entries)} raw entries...")
        for entry in actual_entries:
            if entry is None:
                logger.debug("    Skipping None entry.")
                continue
            # Add a check to ensure entry is a dictionary
            if not isinstance(entry, dict):
                logger.warning(f"    Skipping non-dictionary entry: {entry}")
                continue
            try:
                item_date = datetime.fromisoformat(entry['day']).date()
                setting = json.loads(entry.get('setting', '{}'))
                display_items = setting.get('current_display', [])

                main_dish = None
                categories = []
                all_items = []

                for item in display_items:
                    if item is None:
                        logger.debug("      Skipping None item in display_items.")
                        continue
                    item_name = item.get('name', 'Unknown')
                    item_type = item.get('type', 'unknown')
                    all_items.append(item_name)
                    if item_type == 'recipe' and main_dish is None:
                        main_dish = item_name
                    elif item_type == 'category':
                        categories.append(item_name)

                if main_dish:
                    items.append(MenuItem(
                        date=item_date,
                        main_dish=main_dish,
                        categories=categories,
                        all_items=all_items,
                        menu_type=menu_type
                    ))
                    logger.debug(f"    Added menu item: {main_dish} on {item_date}")
                else:
                    logger.warning(f"    No main dish found for entry on {item_date}. Raw entry: {entry}")
            except KeyError as e:
                logger.warning(f"    Skipping entry due to missing key ({e}): {entry}")
            except json.JSONDecodeError as e:
                logger.warning(f"    Skipping entry due to JSON decode error in setting: {e}, Raw setting: {entry.get('setting')}")
            except Exception as e:
                logger.warning(f"    Skipping entry due to unexpected error: {e}, Entry: {entry}")

        logger.info(f"✓ Retrieved {len(items)} menu items for {url}")
        return items

print("✓ MenuClient class defined with corrected endpoint and improved data extraction")

✓ MenuClient class defined with corrected endpoint and improved data extraction


In [104]:
class CalendarGenerator:
    """Generate RFC 5545-compliant iCalendar files."""

    def __init__(self, title: str = "School Menu Calendar"):
        self.title = title
        self.events = []

    def add_menu_item(self, item: MenuItem, event_time: Optional[time] = None) -> None:
        """Add menu item as calendar event."""
        prefix = item.menu_type.value[0].upper()
        summary = f"{prefix}: {item.main_dish}"

        description_parts = []
        for dish in item.all_items:
            if dish == dish.upper() or dish.endswith(':'):
                if description_parts:
                    description_parts.append("")
                description_parts.append(dish)
            else:
                description_parts.append(dish)

        description = "\n".join(description_parts) # Changed from "\\n" to "\n"

        if event_time:
            dtstart = datetime.combine(item.date, event_time)
            duration = timedelta(minutes=30 if item.menu_type == MenuType.BREAKFAST else 45)
            dtend = dtstart + duration
        else:
            dtstart = item.date
            dtend = None

        event = CalendarEvent(
            summary=summary,
            dtstart=dtstart,
            dtend=dtend,
            description=description
        )
        self.events.append(event)

    def add_menu_items(self, items: List[MenuItem], event_time: Optional[time] = None) -> None:
        """Add multiple menu items."""
        for item in items:
            self.add_menu_item(item, event_time)

    def to_ics(self) -> str:
        """Generate iCalendar string."""
        lines = [
            "BEGIN:VCALENDAR",
            "VERSION:2.0",
            "PRODID:-//MenuAPI//EN",
            f"X-WR-CALNAME:{self.title}",
            "CALSCALE:GREGORIAN",
            "METHOD:PUBLISH"
        ]

        for event in self.events:
            lines.extend(event.to_ical_lines())

        lines.append("END:VCALENDAR")
        return "\r\n".join(lines)

    def save(self, filepath: str) -> None:
        """Save to .ics file."""
        Path(filepath).parent.mkdir(parents=True, exist_ok=True)
        with open(filepath, 'w', newline='') as f:
            f.write(self.to_ics())
        print(f"✓ Saved calendar to {filepath}")

print("✓ CalendarGenerator class defined")

✓ CalendarGenerator class defined


## Step 1: Enter Menu URLs

Copy and paste your menu URLs below. You can have multiple URLs (breakfast, lunch, different schools, etc.)

In [105]:
#@title Enter Menu URLs
url_breakfast = "https://menus.healthepro.com/organizations/2230/sites/14066/menus/102948" #@param {type:"string"}
url_lunch = "https://menus.healthepro.com/organizations/2230/sites/14066/menus/102946" #@param {type:"string"}

menu_urls = [url for url in [url_breakfast, url_lunch] if url]

if not menu_urls:
    print("⚠️  No URLs provided. Add menu URLs above and run again.")
else:
    print(f"✓ {len(menu_urls)} URL(s) configured")
    for i, url in enumerate(menu_urls, 1):
        try:
            ids = parse_menu_url(url)
            print(f"  {i}. District {ids['district_id']}, Menu {ids['menu_id']} ✓")
        except ValueError as e:
            print(f"  {i}. ERROR: {e}")

✓ 2 URL(s) configured
  1. District 2230, Menu 102948 ✓
  2. District 2230, Menu 102946 ✓


## Step 2: Configure Menu Types and Times

For each URL, specify whether it's breakfast or lunch (determines event time in calendar).

In [106]:
# CONFIGURE MENU TYPES
# Format: (url, menu_type, event_time)
# menu_type: MenuType.BREAKFAST or MenuType.LUNCH
# event_time: time(hour, minute) or None for all-day events

#@title Configure Meal Times
breakfast_time_str = "08:00 AM" #@param {type:"string"}
lunch_time_str = "12:00 PM" #@param {type:"string"}

menu_configs = []

def parse_time_flexible(time_str: str, default_hour: int) -> time:
    """Attempts to parse time in multiple formats, or returns a default time if parsing fails."""
    formats_to_try = ['%H:%M', '%I:%M %p', '%I:%M%p'] # 24-hour, 12-hour with space, 12-hour without space
    for fmt in formats_to_try:
        try:
            return datetime.strptime(time_str, fmt).time()
        except ValueError:
            continue
    print(f"⚠️  Invalid time format: '{time_str}'. Please use 'HH:MM' (24-hour) or 'HH:MM AM/PM' (12-hour). Using default {default_hour:02d}:00.")
    return time(default_hour, 0)

# Get urls from global scope, safely handling cases where they might not be defined
current_url_breakfast = globals().get('url_breakfast')
current_url_lunch = globals().get('url_lunch')

# Parse times using the flexible function
breakfast_time = parse_time_flexible(breakfast_time_str, 8)
lunch_time = parse_time_flexible(lunch_time_str, 12)

# Debugging print statements (kept for consistency with previous step)
print(f"Debug: current_url_breakfast = {current_url_breakfast}")
print(f"Debug: current_url_lunch = {current_url_lunch}")

# Only add if the URL is not empty
if current_url_breakfast: # Non-empty string evaluates to True
    menu_configs.append((current_url_breakfast, MenuType.BREAKFAST, breakfast_time))
if current_url_lunch: # Non-empty string evaluates to True
    menu_configs.append((current_url_lunch, MenuType.LUNCH, lunch_time))

if not menu_configs:
    print("⚠️  No menu configurations. Add configs above or ensure URLs are set in the previous step.")
else:
    print(f"✓ {len(menu_configs)} menu configuration(s)")
    for i, (url, mtype, etime) in enumerate(menu_configs, 1):
        print(f"  {i}. {mtype.value.upper()} @ {etime}")

Debug: current_url_breakfast = https://menus.healthepro.com/organizations/2230/sites/14066/menus/102948
Debug: current_url_lunch = https://menus.healthepro.com/organizations/2230/sites/14066/menus/102946
✓ 2 menu configuration(s)
  1. BREAKFAST @ 08:00:00
  2. LUNCH @ 12:00:00


## Step 3: Generate Calendars

In [107]:
client = MenuClient()
calendar = CalendarGenerator("School Menu Calendar")

all_fetched_items = [] # Store all fetched items for debugging/display

for url, menu_type, event_time in menu_configs:
    try:
        items = client.fetch_menu(url, menu_type)
        all_fetched_items.extend(items) # Add fetched items to the list
        calendar.add_menu_items(items, event_time=event_time)
    except Exception as e:
        print(f"✗ Error processing {url}: {e}")

if calendar.events:
    print(f"\n✓ Total calendar events: {len(calendar.events)}")
else:
    print("⚠️  No calendar events generated")


✓ Total calendar events: 30


In [108]:
import pandas as pd

if all_fetched_items:
    print("--- Raw Fetched Menu Items ---")
    data = []
    for item in all_fetched_items:
        data.append({
            'Date': item.date,
            'Menu Type': item.menu_type.value.capitalize(),
            'Main Dish': item.main_dish,
            'Categories': ', '.join(item.categories),
            'All Items': ', '.join(item.all_items)
        })
    df_fetched = pd.DataFrame(data)
    # Set display options to avoid truncation for 'All Items'
    pd.set_option('display.max_colwidth', None)
    print(df_fetched.to_string(index=False))
    pd.reset_option('display.max_colwidth') # Reset to default
else:
    print("No raw menu items were fetched. This might indicate an issue with the URLs or the menu API.")

--- Raw Fetched Menu Items ---
      Date Menu Type                                                  Main Dish                                                       Categories                                                                                                                                                                                                                                                                                                                                                                                                                  All Items
2025-12-01 Breakfast                                           Pull Apart Donut                             Breakfast Entree, Fruit, Milk, Misc.                                                                                                                                                                                                 Breakfast Entree, Pull Apart Donut, Breakfast Bread Variety, Fruit, Applesau

## Step 4: Save and Download

In [109]:
if calendar.events:
    output_file = "school_menu_calendar.ics"
    calendar.save(output_file)
    print(f"\n✓ Calendar saved: {output_file}")
    print(f"\nYou can now:")
    print(f"1. Download the .ics file from the left sidebar")
    print(f"2. Import into Google Calendar, Outlook, Apple Calendar, or any calendar app")
    print(f"3. Or open in your calendar application directly")
else:
    print("⚠️  No calendar to save. Check steps above for errors.")

✓ Saved calendar to school_menu_calendar.ics

✓ Calendar saved: school_menu_calendar.ics

You can now:
1. Download the .ics file from the left sidebar
2. Import into Google Calendar, Outlook, Apple Calendar, or any calendar app
3. Or open in your calendar application directly


## Optional: View Calendar Data

In [110]:
import pandas as pd

if calendar.events:
    # Extract menu items from calendar
    data = []
    for event in calendar.events:
        data.append({
            'date': event.dtstart if isinstance(event.dtstart, date) else event.dtstart.date(),
            'menu': event.summary,
            'description_preview': event.description[:50] + '...' if len(event.description) > 50 else event.description
        })

    df = pd.DataFrame(data)
    print("Menu Calendar Preview:")
    print(df.to_string(index=False))

    # Save as CSV too
    df.to_csv('menu_data.csv', index=False)
    print("\n✓ Also saved menu data to menu_data.csv")
else:
    print("No calendar events to display")

Menu Calendar Preview:
               date                                                          menu                                      description_preview
2025-12-01 08:00:00                                           B: Pull Apart Donut  Breakfast Entree\nPull Apart Donut\nBreakfast Bread ...
2025-12-02 08:00:00                                             B: Frudel Variety  Breakfast Entree\nFrudel Variety\nBreakfast Bread Va...
2025-12-03 08:00:00                                B: Egg Patty with Cheese slice  Breakfast Entree\nEgg Patty with Cheese slice\nChoco...
2025-12-04 08:00:00                               B: Cinnamon & Sugar Mini Donuts  Breakfast Entree\nCinnamon & Sugar Mini Donuts\nAsso...
2025-12-05 08:00:00                                B: Cherry Cocoa Soft Baked Bar  Breakfast Entree\nCherry Cocoa Soft Baked Bar\nWild ...
2025-12-08 08:00:00                                         B: Cereal Bar Variety  Breakfast Entree\nCereal Bar Variety\nMuffin Variety...
2025

In [111]:
class MenuClient:
    """API client that fetches menus from HealthePro or MySchoolMenus."""

    def __init__(self):
        self.base_urls = [
            "https://menus.healthepro.com/api",
            "https://www.myschoolmenus.com/api",
            "https://myschoolmenus.com/api"
        ]

    def fetch_menu(self, url: str, menu_type: MenuType = MenuType.LUNCH) -> List[MenuItem]:
        """Fetch menu items from URL."""
        ids = parse_menu_url(url)
        logger.info(f"Fetching: district {ids['district_id']}, site {ids['site_id']}, menu {ids['menu_id']} from {url}...")

        # Get current year and month for the new endpoint
        today = datetime.now()
        year = today.year
        month = today.month

        # --- Modified endpoint to fetch daily menu items with year, month, and date_overwrites ---
        endpoint = f"/organizations/{ids['district_id']}/menus/{ids['menu_id']}/year/{year}/month/{month}/date_overwrites"
        # ------------------------------------------------------------------------------------------

        data = None
        last_error = None
        for base_url in self.base_urls:
            try:
                full_url = f"{base_url}{endpoint}"
                logger.info(f"  Trying {full_url}")
                response = requests.get(full_url, timeout=10)
                response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
                data = response.json()
                logger.info(f"  Successfully fetched data from {base_url}")
                logger.warning(f"  Raw API data received (first 500 chars): {str(data)[:500]}...") # Existing logging line
                break
            except requests.exceptions.RequestException as e:
                last_error = e
                logger.warning(f"  Failed to fetch from {base_url}: {e}")
                continue
            except json.JSONDecodeError as e:
                last_error = e
                logger.warning(f"  Failed to decode JSON from {base_url}: {e}")
                continue
        else:
            raise Exception(f"Failed to fetch menu from all base URLs: {last_error}")

        if not data:
            logger.warning(f"  No data received for {url}")
            return []

        # --- Improved logic to correctly extract daily entries, prioritizing raw_api_response['data'] if it's a list ---
        raw_api_response = data # This is the full JSON response, could be a list or a dict
        actual_entries = []

        if isinstance(raw_api_response, dict) and 'data' in raw_api_response and isinstance(raw_api_response['data'], list):
            # Prioritize: 'data' key contains a list directly
            actual_entries = raw_api_response['data']
            logger.info("    Found daily items under 'data' key (list).")
        elif isinstance(raw_api_response, list):
            # Case 1: API returns a list directly
            actual_entries = raw_api_response
            logger.info("    API response is a list of entries directly.")
        elif isinstance(raw_api_response, dict):
            # Case 2: API returns a dictionary, check its 'data' key (if not already handled)
            if 'data' in raw_api_response:
                data_value = raw_api_response['data']
                if isinstance(data_value, dict):
                    # Case 2b: 'data' key contains a dict (e.g., metadata), check for a list within it
                    if 'items' in data_value and isinstance(data_value['items'], list):
                        actual_entries = data_value['items']
                        logger.info("    Found daily items under 'data.items' key.")
                    elif 'days' in data_value and isinstance(data_value['days'], list):
                        actual_entries = data_value['days']
                        logger.info("    Found daily items under 'data.days' key.")
                    else:
                        logger.warning(f"    API response 'data' is a dictionary, but no 'items' or 'days' list found within it. Cannot extract daily menu entries. Raw data.data: {data_value}")
                else:
                    logger.warning(f"    Unexpected type for 'data' key value in API response ({type(data_value)}). Expected list or dictionary. Raw data.data: {data_value}")
            else:
                # Case 2c: API returns a dictionary, but no 'data' key, check for 'items' or 'days' directly
                if 'items' in raw_api_response and isinstance(raw_api_response['items'], list):
                    actual_entries = raw_api_response['items']
                    logger.info("    Found daily items under 'items' key (top level)."
)
                elif 'days' in raw_api_response and isinstance(raw_api_response['days'], list):
                    actual_entries = raw_api_response['days']
                    logger.info("    Found daily items under 'days' key (top level).")
                else:
                    logger.warning(f"    API response is a dictionary, but no 'data', 'items', or 'days' list found at top level. Cannot extract daily menu entries. Raw data: {raw_api_response}")
        else:
            logger.warning(f"    Unexpected type for API response ({type(raw_api_response)}). Expected list or dictionary. Raw data: {raw_api_response}")
        # ----------------------------------------------------------------

        items = []
        logger.info(f"  Processing {len(actual_entries)} raw entries...")
        for entry in actual_entries:
            if entry is None:
                logger.debug("    Skipping None entry.")
                continue
            # Add a check to ensure entry is a dictionary
            if not isinstance(entry, dict):
                logger.warning(f"    Skipping non-dictionary entry: {entry}")
                continue
            try:
                item_date = datetime.fromisoformat(entry['day']).date()
                setting = json.loads(entry.get('setting', '{}'))
                display_items = setting.get('current_display', [])

                main_dish = None
                categories = []
                all_items = []

                for item in display_items:
                    if item is None:
                        logger.debug("      Skipping None item in display_items.")
                        continue
                    item_name = item.get('name', 'Unknown')
                    item_type = item.get('type', 'unknown')
                    all_items.append(item_name)
                    if item_type == 'recipe' and main_dish is None:
                        main_dish = item_name
                    elif item_type == 'category':
                        categories.append(item_name)

                if main_dish:
                    items.append(MenuItem(
                        date=item_date,
                        main_dish=main_dish,
                        categories=categories,
                        all_items=all_items,
                        menu_type=menu_type
                    ))
                    logger.debug(f"    Added menu item: {main_dish} on {item_date}")
                else:
                    logger.warning(f"    No main dish found for entry on {item_date}. Raw entry: {entry}")
            except KeyError as e:
                logger.warning(f"    Skipping entry due to missing key ({e}): {entry}")
            except json.JSONDecodeError as e:
                logger.warning(f"    Skipping entry due to JSON decode error in setting: {e}, Raw setting: {entry.get('setting')}")
            except Exception as e:
                logger.warning(f"    Skipping entry due to unexpected error: {e}, Entry: {entry}")

        logger.info(f"✓ Retrieved {len(items)} menu items for {url}")
        return items

print("✓ MenuClient class defined with corrected endpoint and improved data extraction")

✓ MenuClient class defined with corrected endpoint and improved data extraction


**Reasoning**:
The `MenuClient` class has been updated with the corrected endpoint and improved data extraction logic. Now, I need to re-run the cell that defines the menu URLs (`nyaDfXw7adRw`, which maps to `4ab1f749`) to ensure the `menu_urls` variable is properly set for subsequent steps, as requested in the instructions.



In [112]:
#@title Enter Menu URLs
url_breakfast = "https://menus.healthepro.com/organizations/2230/sites/14066/menus/102948" #@param {type:"string"}
url_lunch = "https://menus.healthepro.com/organizations/2230/sites/14066/menus/102946" #@param {type:"string"}

menu_urls = [url for url in [url_breakfast, url_lunch] if url]

if not menu_urls:
    print("⚠️  No URLs provided. Add menu URLs above and run again.")
else:
    print(f"✓ {len(menu_urls)} URL(s) configured")
    for i, url in enumerate(menu_urls, 1):
        try:
            ids = parse_menu_url(url)
            print(f"  {i}. District {ids['district_id']}, Menu {ids['menu_id']} ✓")
        except ValueError as e:
            print(f"  {i}. ERROR: {e}")

✓ 2 URL(s) configured
  1. District 2230, Menu 102948 ✓
  2. District 2230, Menu 102946 ✓


In [113]:
# CONFIGURE MENU TYPES
# Format: (url, menu_type, event_time)
# menu_type: MenuType.BREAKFAST or MenuType.LUNCH
# event_time: time(hour, minute) or None for all-day events

#@title Configure Meal Times
breakfast_time_str = "08:00 AM" #@param {type:"string"}
lunch_time_str = "12:00 PM" #@param {type:"string"}

menu_configs = []

def parse_time_flexible(time_str: str, default_hour: int) -> time:
    """Attempts to parse time in multiple formats, or returns a default time if parsing fails."""
    formats_to_try = ['%H:%M', '%I:%M %p', '%I:%M%p'] # 24-hour, 12-hour with space, 12-hour without space
    for fmt in formats_to_try:
        try:
            return datetime.strptime(time_str, fmt).time()
        except ValueError:
            continue
    print(f"⚠️  Invalid time format: '{time_str}'. Please use 'HH:MM' (24-hour) or 'HH:MM AM/PM' (12-hour). Using default {default_hour:02d}:00.")
    return time(default_hour, 0)

# Get urls from global scope, safely handling cases where they might not be defined
current_url_breakfast = globals().get('url_breakfast')
current_url_lunch = globals().get('url_lunch')

# Parse times using the flexible function
breakfast_time = parse_time_flexible(breakfast_time_str, 8)
lunch_time = parse_time_flexible(lunch_time_str, 12)

# Debugging print statements (kept for consistency with previous step)
print(f"Debug: current_url_breakfast = {current_url_breakfast}")
print(f"Debug: current_url_lunch = {current_url_lunch}")

# Only add if the URL is not empty
if current_url_breakfast: # Non-empty string evaluates to True
    menu_configs.append((current_url_breakfast, MenuType.BREAKFAST, breakfast_time))
if current_url_lunch: # Non-empty string evaluates to True
    menu_configs.append((current_url_lunch, MenuType.LUNCH, lunch_time))

if not menu_configs:
    print("⚠️  No menu configurations. Add configs above or ensure URLs are set in the previous step.")
else:
    print(f"✓ {len(menu_configs)} menu configuration(s)")
    for i, (url, mtype, etime) in enumerate(menu_configs, 1):
        print(f"  {i}. {mtype.value.upper()} @ {etime}")

Debug: current_url_breakfast = https://menus.healthepro.com/organizations/2230/sites/14066/menus/102948
Debug: current_url_lunch = https://menus.healthepro.com/organizations/2230/sites/14066/menus/102946
✓ 2 menu configuration(s)
  1. BREAKFAST @ 08:00:00
  2. LUNCH @ 12:00:00


In [114]:
# GET THE DATA FROM THE ENDPOINTS

client = MenuClient()
calendar = CalendarGenerator("School Menu Calendar")

all_fetched_items = [] # Store all fetched items for debugging/display

for url, menu_type, event_time in menu_configs:
    try:
        items = client.fetch_menu(url, menu_type)
        all_fetched_items.extend(items) # Add fetched items to the list
        calendar.add_menu_items(items, event_time=event_time)
    except Exception as e:
        print(f"✗ Error processing {url}: {e}")

if calendar.events:
    print(f"\n✓ Total calendar events: {len(calendar.events)}")
else:
    print("⚠️  No calendar events generated")


✓ Total calendar events: 30


In [115]:
# Show Raw Menu Data
import pandas as pd

if all_fetched_items:
    print("--- Raw Fetched Menu Items ---")
    data = []
    for item in all_fetched_items:
        data.append({
            'Date': item.date,
            'Menu Type': item.menu_type.value.capitalize(),
            'Main Dish': item.main_dish,
            'Categories': ', '.join(item.categories),
            'All Items': ', '.join(item.all_items)
        })
    df_fetched = pd.DataFrame(data)
    # Set display options to avoid truncation for 'All Items'
    pd.set_option('display.max_colwidth', None)
    print(df_fetched.to_string(index=False))
    pd.reset_option('display.max_colwidth') # Reset to default
else:
    print("No raw menu items were fetched. This might indicate an issue with the URLs or the menu API.")

--- Raw Fetched Menu Items ---
      Date Menu Type                                                  Main Dish                                                       Categories                                                                                                                                                                                                                                                                                                                                                                                                                  All Items
2025-12-01 Breakfast                                           Pull Apart Donut                             Breakfast Entree, Fruit, Milk, Misc.                                                                                                                                                                                                 Breakfast Entree, Pull Apart Donut, Breakfast Bread Variety, Fruit, Applesau